In [1]:
import pandas as pd
merged_cmu = pd.read_csv("merge_cmu.csv")
merged_cmu

,movie_name,movie_date,box_office,runtime,language,countries,genres,author_book,year_book
0,marypoppins,1964,102272727.0,139.0,English Language,United States of America,"Children's/Family, Musical, Fantasy, Comedy, D...",P. L. Travers,1934–1988
1,mysteriousisland,1982,NaN,100.0,Standard Mandarin,Hong Kong,"Action/Adventure, Wuxia, Martial Arts Film, Ch...",Jules Verne,1874
2,juarez,1939,NaN,125.0,"English Language, Spanish Language",United States of America,"Costume drama, Biographical film, Historical f...",Bertita Harding,1934
3,thegreatsantini,1979,4702575.0,115.0,English Language,United States of America,"Family Drama, Drama",Pat Conroy,1976
4,thecastle,1968,NaN,88.0,German Language,West Germany,"Mystery, Drama",Franz Kafka,1926
...,...,...,...,...,...,...,...,...,...
2321,ivanhoe,1913,NaN,NaN,English Language,United Kingdom,"Silent film, Drama, Historical drama",Sir Walter Scott,1820
2322,ivanhoe,1913,NaN,NaN,English Language,United Kingdom,"Silent film, Drama, Historical drama",Sir Walter Scott,1820
2323,risingsun,1993,107198790.0,129.0,English Language,United States of America,"Thriller, Crime Fiction, Mystery, Drama, Suspe...",Michael Crichton,1992
2324,thedeluge,1974,NaN,315.0,Polish Language,Poland,War film,Henryk Sienkiewicz,1886


In [2]:
data_folder = "data/"

def clean_title(title):
    return title.lower().replace(" ", "")

def merge_imdb_id(df):
    titles = pd.read_csv(data_folder + "title.basics.tsv", sep='\t', header=0, usecols=[0, 1, 2, 3, 4, 5], names=['imdbID', 'titleType', 'imdbPrimaryTitle', 'imdbOriginalTitle', 'isAdult', 'imdbYear'], dtype={'isAdult': 'string'})
    
    titles['imdbYear'] = pd.to_numeric(titles['imdbYear'], errors='coerce')
    titles['isAdult'] = pd.to_numeric(titles['isAdult'], errors='coerce')
    titles = titles.dropna()
    titles = titles[titles['titleType'] == 'movie']
    
    titles['imdbOriginalTitle'] = titles['imdbOriginalTitle'].apply(clean_title)
    titles['imdbPrimaryTitle'] = titles['imdbPrimaryTitle'].apply(clean_title)
    mergeOnOriginal = pd.merge(titles, df, how='inner', left_on=['imdbOriginalTitle', 'imdbYear'], right_on=['movie_name', 'movie_date'])
    mergeOnPrimary = pd.merge(titles, df, how='inner', left_on=['imdbPrimaryTitle', 'imdbYear'], right_on=['movie_name', 'movie_date'])
    
    merge = pd.concat([mergeOnOriginal, mergeOnPrimary], axis=0)
    merge = merge.drop_duplicates(subset=['imdbID'])
    merge = merge.drop(['imdbPrimaryTitle', 'imdbOriginalTitle', 'imdbYear', 'titleType'], axis=1)
    
    return merge

merged_imdb_id = merge_imdb_id(merged_cmu)
merged_imdb_id

,imdbID,isAdult,movie_name,movie_date,box_office,runtime,language,countries,genres,author_book,year_book
0,tt0000679,0,thefairylogueandradio-plays,1908,NaN,120.0,English Language,United States of America,"Silent film, Black-and-white",L. Frank Baum,1900
1,tt0002646,0,atlantis,1913,NaN,113.0,"English Language, Danish Language",Denmark,"Silent film, Drama, Indie, Black-and-white",Gerhart Hauptmann,1912
2,tt0003022,0,ivanhoe,1913,NaN,NaN,"Silent film, English Language",United States of America,"Swashbuckler films, Silent film, Drama, Adventure",Sir Walter Scott,1820
6,tt0004099,0,"hismajesty,thescarecrowofoz",1914,NaN,60.0,English Language,United States of America,"Silent film, Adventure, Children's/Family, Bla...",L. Frank Baum,1900
7,tt0004182,0,thejungle,1914,NaN,NaN,"Silent film, English Language",NaN,Silent film,Upton Sinclair,1906
...,...,...,...,...,...,...,...,...,...,...,...
1956,tt1736633,0,"oslo,august31st",2011,NaN,95.0,Norwegian Language,Norway,Drama,Pierre Drieu La Rochelle,1931
1965,tt2063013,0,theassassins,2012,NaN,NaN,Standard Mandarin,China,Drama,Luo Guanzhong,NaN
1966,tt2071441,0,dangerousliaisons,2012,NaN,110.0,"English Language, Standard Mandarin","Singapore, South Korea, China","Mystery, Romance Film, Drama",Pierre Choderlos de Laclos,1782
1968,tt2308725,0,helpless,2012,16175929.0,117.0,Korean Language,South Korea,"Thriller, Mystery",Miyuki Miyabe,1992


In [3]:
def merge_imdb_ratings(df):
    imdb_ratings = pd.read_csv(data_folder + "title.ratings.tsv", sep='\t', header=0, names=['imdbID', 'rating', 'numVotes'])
    
    imdb_ratings['rating'] = pd.to_numeric(imdb_ratings['rating'])
    imdb_ratings['numVotes'] = pd.to_numeric(imdb_ratings['numVotes'])
    imdb_ratings = imdb_ratings.dropna()

    return pd.merge(df, imdb_ratings, how='inner', on=['imdbID'])

merged_imdb = merge_imdb_ratings(merged_imdb_id)
merged_imdb

,imdbID,isAdult,movie_name,movie_date,box_office,runtime,language,countries,genres,author_book,year_book,rating,numVotes
0,tt0000679,0,thefairylogueandradio-plays,1908,NaN,120.0,English Language,United States of America,"Silent film, Black-and-white",L. Frank Baum,1900,5.2,76
1,tt0002646,0,atlantis,1913,NaN,113.0,"English Language, Danish Language",Denmark,"Silent film, Drama, Indie, Black-and-white",Gerhart Hauptmann,1912,6.5,500
2,tt0003022,0,ivanhoe,1913,NaN,NaN,"Silent film, English Language",United States of America,"Swashbuckler films, Silent film, Drama, Adventure",Sir Walter Scott,1820,5.6,97
3,tt0004099,0,"hismajesty,thescarecrowofoz",1914,NaN,60.0,English Language,United States of America,"Silent film, Adventure, Children's/Family, Bla...",L. Frank Baum,1900,5.3,533
4,tt0004182,0,thejungle,1914,NaN,NaN,"Silent film, English Language",NaN,Silent film,Upton Sinclair,1906,6.8,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1935,tt1736633,0,"oslo,august31st",2011,NaN,95.0,Norwegian Language,Norway,Drama,Pierre Drieu La Rochelle,1931,7.6,30513
1936,tt2063013,0,theassassins,2012,NaN,NaN,Standard Mandarin,China,Drama,Luo Guanzhong,NaN,5.5,2147
1937,tt2071441,0,dangerousliaisons,2012,NaN,110.0,"English Language, Standard Mandarin","Singapore, South Korea, China","Mystery, Romance Film, Drama",Pierre Choderlos de Laclos,1782,6.0,2092
1938,tt2308725,0,helpless,2012,16175929.0,117.0,Korean Language,South Korea,"Thriller, Mystery",Miyuki Miyabe,1992,6.7,2642


In [5]:
print('lines dropped during merge with IMDB: ', len(merged_cmu) - len(merged_imdb))
merged_imdb.to_csv('merge_imdb.csv', index=False)
merged_imdb

lines dropped during merge with IMDB:  386


,imdbID,isAdult,movie_name,movie_date,box_office,runtime,language,countries,genres,author_book,year_book,rating,numVotes
0,tt0000679,0,thefairylogueandradio-plays,1908,NaN,120.0,English Language,United States of America,"Silent film, Black-and-white",L. Frank Baum,1900,5.2,76
1,tt0002646,0,atlantis,1913,NaN,113.0,"English Language, Danish Language",Denmark,"Silent film, Drama, Indie, Black-and-white",Gerhart Hauptmann,1912,6.5,500
2,tt0003022,0,ivanhoe,1913,NaN,NaN,"Silent film, English Language",United States of America,"Swashbuckler films, Silent film, Drama, Adventure",Sir Walter Scott,1820,5.6,97
3,tt0004099,0,"hismajesty,thescarecrowofoz",1914,NaN,60.0,English Language,United States of America,"Silent film, Adventure, Children's/Family, Bla...",L. Frank Baum,1900,5.3,533
4,tt0004182,0,thejungle,1914,NaN,NaN,"Silent film, English Language",NaN,Silent film,Upton Sinclair,1906,6.8,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1935,tt1736633,0,"oslo,august31st",2011,NaN,95.0,Norwegian Language,Norway,Drama,Pierre Drieu La Rochelle,1931,7.6,30513
1936,tt2063013,0,theassassins,2012,NaN,NaN,Standard Mandarin,China,Drama,Luo Guanzhong,NaN,5.5,2147
1937,tt2071441,0,dangerousliaisons,2012,NaN,110.0,"English Language, Standard Mandarin","Singapore, South Korea, China","Mystery, Romance Film, Drama",Pierre Choderlos de Laclos,1782,6.0,2092
1938,tt2308725,0,helpless,2012,16175929.0,117.0,Korean Language,South Korea,"Thriller, Mystery",Miyuki Miyabe,1992,6.7,2642
